In [38]:
import pandas as pd
import pickle
from scipy.stats import poisson


In [39]:
dict_tables = pickle.load(open('dict_tables','rb'))
df_historical_data = pd.read_csv('UEFA_Euro_Historical_data_cleaned.csv')
df_fixture = pd.read_csv('UEFA_Euro_Fixtures_cleaned.csv')

Calculating Team Strength

In [40]:
df_home = df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away = df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [41]:
df_home = df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team','AwayGoals':'GoalsScored','HomeGoals':'GoalsConceded'})

In [42]:


# Concatenate the DataFrames
concatenated_df = pd.concat([df_home, df_away], ignore_index=False)

# Group by 'Team' and calculate mean
df_team_strength = concatenated_df.groupby('Team').mean()
print(df_team_strength.columns)
print(df_team_strength)

df_team_strength.reset_index(inplace=True)

# access the 'Team' column among the columns
print(df_team_strength.columns)
k = df_team_strength 
# Assuming 'Team' is the column you want to make lowercase
df_team_strength['Team'] = df_team_strength['Team'].str.lower()

df_team_strength['Team']=df_team_strength['Team'].str.strip()
df_team_strength.to_csv('teamstrength.csv')



Index(['GoalsScored', 'GoalsConceded'], dtype='object')
               GoalsScored  GoalsConceded
Team                                     
Albania           0.000000       1.000000
Austria           0.800000       1.200000
Belgium           1.545455       1.000000
Bulgaria          0.500000       1.000000
CIS               1.000000       1.000000
...                    ...            ...
 Ukraine          1.000000       1.750000
 Wales            1.200000       1.000000
 West Germany     1.833333       1.000000
 Yugoslavia       1.666667       3.166667
  Switzerland     0.875000       1.625000

[80 rows x 2 columns]
Index(['Team', 'GoalsScored', 'GoalsConceded'], dtype='object')


function to predict points


In [43]:
dict_tables['Group A']['Team'][0] = 'Germany'

C:\Users\Aditya\AppData\Local\Temp\ipykernel_20068\108545762.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dict_tables['Group A']['Team'][0] = 'Germany'
C:\Users\Aditya\AppData\Local\Temp\ipykernel_20068\108545762.py:1: SettingWithCopy

In [44]:

# Define the function to predict points
def predict_points(home, away, df_team_strength_data):
    home = home.strip().lower()  # Strip leading/trailing whitespaces and convert to lowercase
    away = away.strip().lower()  # Strip leading/trailing whitespaces and convert to lowercase
    
    if home in df_team_strength.index and away in df_team_strength.index:

        homeScored = df_team_strength.loc[home, 'GoalsScored']
        awayConceded = df_team_strength.loc[away, 'GoalsConceded']
        awayScored = df_team_strength.loc[away, 'GoalsScored']
        homeConceded = df_team_strength.loc[home, 'GoalsConceded']
        
        # Calculate lambda values for Poisson distribution
        lamb_home = homeScored * awayConceded
        lamb_away = awayScored * homeConceded
        
        prob_home, prob_away, prob_draw = 0, 0, 0
        
        # Calculate probabilities for different outcomes
        for x in range(0, 11):  # number of goals for home team
            for y in range(0, 11):  # number of goals for away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                elif x < y:
                    prob_away += p
        
        # Calculate points for home and away teams
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        
        return (points_home, points_away)
    else:
        # If either home or away team is not in df_team_strength index
        return (0, 0)

# Provided DataFrame
df_team_strength_data = """
Team,GoalsScored,GoalsConceded
albania,0.25,1.0
austria,0.7,1.2
belgium,1.4090909090909092,1.2727272727272727
bulgaria,0.625,1.875
cis,0.5,1.25
croatia,1.3416666666666666,1.225
czech republic,1.233173076923077,1.2788461538461537
czechoslovakia,1.6666666666666667,1.1666666666666667
denmark,1.2720588235294117,1.5161764705882353
england,1.3333333333333333,0.9814814814814815
fr yugoslavia,2.0,3.25
finland,0.5,0.75
france,1.520125,1.165
germany,1.6,1.0
greece,0.9047619047619048,1.1666666666666667
hungary,1.3214285714285714,1.1964285714285714
iceland,1.5833333333333333,1.1666666666666667
italy,0.64,0.72
latvia,0.25,1.25
netherlands,1.65 , 1.041
north macedonia,0.5,2.25
northern ireland,0.3333333333333333,0.6666666666666666
norway,0.25,0.25
poland,0.7916666666666666,1.0416666666666667
portugal,1.4342105263157894,1.0526315789473684
republic of ireland,0.625,1.5833333333333333
romania,0.7118055555555556,1.2638888888888888
russia,1.1,1.8
scotland,0.525,1.2
slovakia,0.5,2.05
slovenia,1.5,3.0
soviet union,1.3571428571428572,1.0714285714285714
spain,1.5714285714285714,1.0714285714285714
sweden,1.125,1.2857142857142858
switzerland,0.9625,1.1375
turkey,0.85,1.6
ukraine,0.6666666666666666,1.6666666666666667
wales,1.4,1.2
"""

# Convert provided data to DataFrame
data = [line.split(',') for line in df_team_strength_data.strip().split('\n')]
columns = data[0]
data = data[1:]
df_team_strength = pd.DataFrame(data, columns=columns)
df_team_strength.set_index('Team', inplace=True)
df_team_strength = df_team_strength.astype(float)


In [45]:
home_team ='Germany'
away_team ='Switzerland'
points_home, points_away = predict_points(home_team, away_team, df_team_strength)
print("Predicted points for", home_team, ":", points_home)
print("Predicted points for", away_team, ":", points_away)

Predicted points for Germany : 1.9536979151722274
Predicted points for Switzerland : 0.818202506526779


In [46]:
df_fixture_36=df_fixture[:36].copy()
df_fixture_knockout=df_fixture[36:44].copy()
df_fixture_quater=df_fixture[44:48].copy()
df_fixture_semi=df_fixture[48:50].copy()
df_fixture_final=df_fixture[50:].copy()


In [47]:
for group in dict_tables:
    teams_in_group = dict_tables[group]
    df_fixture_group_6 = df_fixture_36[df_fixture_36['home'].isin(teams_in_group['Team'])]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away, df_team_strength)  # Pass df_team_strength
        dict_tables[group].loc[dict_tables[group]['Team']==home, 'Pts'] += points_home
        dict_tables[group].loc[dict_tables[group]['Team']==away, 'Pts'] += points_away
        
    dict_tables[group] = dict_tables[group].sort_values('Pts', ascending=False).reset_index()
    dict_tables[group] = dict_tables[group][['Team', 'Pts']]
    dict_tables[group] = dict_tables[group].round(0)
    



C:\Users\Aditya\AppData\Local\Temp\ipykernel_20068\3865007464.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.32567973]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tables[group].loc[dict_tables[group]['Team']==home, 'Pts'] += points_home
C:\Users\Aditya\AppData\Local\Temp\ipykernel_20068\3865007464.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.69226237]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tables[group].loc[dict_tables[group]['Team']==home, 'Pts'] += points_home
C:\Users\Aditya\AppData\Local\Temp\ipykernel_20068\3865007464.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.73756209]' has dtype incompatible with int

In [48]:
df_fixture_36

,home,score,away,year
0,Germany,Match 1,Scotland,2024
1,Hungary,Match 2,Switzerland,2024
2,Germany,Match 14,Hungary,2024
3,Scotland,Match 13,Switzerland,2024
4,Switzerland,Match 25,Germany,2024
5,Scotland,Match 26,Hungary,2024
6,Spain,Match 3,Croatia,2024
7,Italy,Match 4,Albania,2024
8,Croatia,Match 15,Albania,2024
9,Spain,Match 16,Italy,2024


In [49]:
df_fixture_knockout

,home,score,away,year
36,Runner-up Group A,Match 38,Runner-up Group B,2024
37,Winner Group A,Match 37,Runner-up Group C,2024
38,Winner Group C,Match 40,3rd Group D/E/F,2024
39,Winner Group B,Match 39,3rd Group A/D/E/F,2024
40,Runner-up Group D,Match 42,Runner-up Group E,2024
41,Winner Group F,Match 41,3rd Group A/B/C,2024
42,Winner Group E,Match 43,3rd Group A/B/C/D,2024
43,Winner Group D,Match 44,Runner-up Group F,2024


In [50]:
df_fixture_quater

,home,score,away,year
44,Winner Match 39,Match 45,Winner Match 37,2024
45,Winner Match 41,Match 46,Winner Match 42,2024
46,Winner Match 40,Match 48,Winner Match 38,2024
47,Winner Match 43,Match 47,Winner Match 44,2024


In [51]:
df_fixture_semi

,home,score,away,year
48,Winner Match 45,Match 49,Winner Match 46,2024
49,Winner Match 47,Match 50,Winner Match 48,2024


In [52]:
df_fixture_final

,home,score,away,year
50,Winner Match 49,Match 51,Winner Match 50,2024


In [53]:
dict_tables['Group A']

,Team,Pts
0,Germany,6.0
1,Hungary,5.0
2,Switzerland,4.0
3,Scotland,2.0


In [54]:
for group in dict_tables:
     print(dict_tables[group]['Team'].values)

['Germany' 'Hungary' 'Switzerland' 'Scotland']
['Spain' 'Croatia' 'Italy' 'Albania']
['England' 'Denmark' 'Slovenia' 'Serbia']
['Netherlands' 'France' 'Poland' 'Austria']
['Belgium' 'Romania' 'Ukraine' 'Slovakia']
['Portugal' 'Czech Republic' 'Turkey' 'Georgia']


In [55]:
for group in dict_tables:
    teams_in_group = dict_tables[group]
    df_fixture_group_6 = df_fixture_36[df_fixture_36['home'].isin(teams_in_group['Team'])]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away, df_team_strength)  # Pass df_team_strength
        dict_tables[group].loc[dict_tables[group]['Team']==home, 'Pts'] += points_home
        dict_tables[group].loc[dict_tables[group]['Team']==away, 'Pts'] += points_away
        
    dict_tables[group] = dict_tables[group].sort_values('Pts', ascending=False).reset_index()
    dict_tables[group] = dict_tables[group][['Team', 'Pts']]
    dict_tables[group] = dict_tables[group].round(0)


In [56]:
dict_tables['Group A']

,Team,Pts
0,Germany,12.0
1,Hungary,10.0
2,Switzerland,8.0
3,Scotland,4.0


In [57]:
for group in dict_tables:
       print (dict_tables[group])

          Team   Pts
0      Germany  12.0
1      Hungary  10.0
2  Switzerland   8.0
3     Scotland   4.0
      Team   Pts
0    Spain  12.0
1  Croatia  10.0
2    Italy   8.0
3  Albania   4.0
       Team  Pts
0   England  8.0
1   Denmark  6.0
2  Slovenia  2.0
3    Serbia  0.0
          Team   Pts
0  Netherlands  12.0
1       France  10.0
2       Poland   6.0
3      Austria   4.0
       Team   Pts
0   Belgium  14.0
1   Romania   8.0
2   Ukraine   6.0
3  Slovakia   4.0
             Team  Pts
0        Portugal  8.0
1  Czech Republic  6.0
2          Turkey  2.0
3         Georgia  0.0


In [58]:
def get_top_two_third_place_teams(tournament_data):
    third_place_teams = []
    for group, teams in tournament_data.items():
        if len(teams) >= 3:  # Ensure there are at least 3 teams in the group
            sorted_teams = sorted(teams, key=lambda x: x[1], reverse=True)
            if len(sorted_teams) >= 3:
                third_place_teams.append(sorted_teams[2][0])  # Append only the team name
    return sorted(third_place_teams)[:4]  # Sort team names and return top two

# Sample data (provided by you)
tournament_data = {
    "Group A": [("Germany", 5.0), ("Hungary", 5.0), ("Switzerland", 4.0), ("Scotland", 2.0)],
    "Group B": [("Spain", 6.0), ("Croatia", 5.0), ("Italy", 4.0), ("Albania", 2.0)],
    "Group C": [("England", 4.0), ("Denmark", 3.0), ("Slovenia", 1.0), ("Serbia", 0.0)],
    "Group D": [("Netherlands", 6.0), ("France", 5.0), ("Poland", 3.0), ("Austria", 2.0)],
    "Group E": [("Belgium", 7.0), ("Romania", 4.0), ("Ukraine", 3.0), ("Slovakia", 2.0)],
    "Group F": [("Portugal", 4.0), ("Czech Republic", 1.0), ("Turkey", 1.0), ("Georgia", 0.0)]
}

top_two_third_place_teams = get_top_two_third_place_teams(tournament_data)
for team in top_two_third_place_teams:
    print(team)
    
    #B 41 italy
    #D 38 Poland
    #C 42 Slovenia
    #A 39 Switzerland


Italy
Poland
Slovenia
Switzerland


In [59]:
dict_tables["Group B"].loc[0,'Team']

'Spain'

In [60]:
# update the knock out fixture and runners up 
for group in dict_tables:
    group_winner = dict_tables[group].loc[0,'Team']
    runners_up = dict_tables[group].loc[1,'Team']
    print(group)
    df_fixture_knockout.replace({f'Winner {group}':group_winner,
                                f'Runner-up {group}':runners_up}, inplace=True)
    
    df_fixture_knockout.replace({'3rd Group D/E/F':'Poland',
                                f'3rd Group A/D/E/F':'Switzerland',
                                '3rd Group A/B/C':'Italy',
                                '3rd Group A/B/C/D':'Slovenia'}, inplace=True)
    
    
df_fixture_knockout['winner']='?'
    

Group A
Group B
Group C
Group D
Group E
Group F


Knockouts

In [61]:
def get_winner(df_fixture_updated):
    for index ,row in df_fixture_updated.iterrows():
        home,away = row['home'] , row['away']
        points_home ,points_away =predict_points(home,away,df_team_strength_data=df_team_strength)
        if points_home > points_away :
            winner = home
        else :
            winner = away 
        
        df_fixture_updated.loc[index,'winner']=winner
        
    return df_fixture_updated

In [62]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
36,Hungary,Match 38,Croatia,2024,Hungary
37,Germany,Match 37,Denmark,2024,Germany
38,England,Match 40,Poland,2024,England
39,Spain,Match 39,Switzerland,2024,Spain
40,France,Match 42,Romania,2024,France
41,Portugal,Match 41,Italy,2024,Portugal
42,Belgium,Match 43,Slovenia,2024,Belgium
43,Netherlands,Match 44,Czech Republic,2024,Netherlands


In [63]:
df_fixture_knockout.columns


Index(['home', 'score', 'away', 'year', 'winner'], dtype='object')

In [64]:
def update_table(df_fixture_round_1, df_fixture_round_2):

    for index, row in df_fixture_round_1.iterrows():
        winner = row['winner']  # Assuming 'winner' is a column in df_fixture_round_1
        match = row['score']  # Assuming 'score' is a column in df_fixture_round_1
        df_fixture_round_2.replace({f'Winner {match}': winner}, inplace=True)

    df_fixture_round_2['winner'] = '?'


Quater final


In [65]:
update_table(df_fixture_knockout,df_fixture_quater)


In [66]:
df_fixture_quater

,home,score,away,year,winner
44,Spain,Match 45,Germany,2024,?
45,Portugal,Match 46,France,2024,?
46,England,Match 48,Hungary,2024,?
47,Belgium,Match 47,Netherlands,2024,?


In [67]:
get_winner(df_fixture_quater)

,home,score,away,year,winner
44,Spain,Match 45,Germany,2024,Germany
45,Portugal,Match 46,France,2024,Portugal
46,England,Match 48,Hungary,2024,England
47,Belgium,Match 47,Netherlands,2024,Netherlands


In [68]:
update_table(df_fixture_quater,df_fixture_semi)

In [69]:
df_fixture_semi

,home,score,away,year,winner
48,Germany,Match 49,Portugal,2024,?
49,Netherlands,Match 50,England,2024,?


In [70]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
48,Germany,Match 49,Portugal,2024,Germany
49,Netherlands,Match 50,England,2024,Netherlands


In [71]:
update_table(df_fixture_semi,df_fixture_final)

In [72]:
df_fixture_final

,home,score,away,year,winner
50,Germany,Match 51,Netherlands,2024,?


In [73]:
get_winner(df_fixture_final)

,home,score,away,year,winner
50,Germany,Match 51,Netherlands,2024,Germany


In [74]:
df_fixture_final['winner']

50    Germany
Name: winner, dtype: object